<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

In [1]:
import pandas as pd
import networkx as nx
import numpy as np

<html><font size=4><B>1. Κατασκευή γράφου και υπολογισμός μετρικών</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase) </a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$. Οι κόμβοι του γράφου αναπαριστούν πρωτεΐνες και οι ακμές την αλληλεπίδραση αυτών. Στην σελίδα <a href="http://konect.cc/networks/maayan-pdzbase/"> KONECT (PDZBase)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί.<BR></html>

In [3]:
datafile = "out.maayan-pdzbase"
df = pd.read_csv(datafile, sep='\t', header=None, names=['source', 'target'])

df = df[df['source'] != df['target']] # Αφαίρεση self-loops

df['source'], df['target'] = (
    df[['source', 'target']].min(axis=1), # Ο μικρότερος κόμβος γίνεται "source"
    df[['source', 'target']].max(axis=1)  # Ο μεγαλύτερος κόμβος γίνεται "target"
)

df = df.drop_duplicates()  # Αφαίρεση διπλότυπων ακμών

G = nx.from_pandas_edgelist(df, source='source', target='target') # Μετατροπή σε γράφο

 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

In [4]:
initial_node_count=len(G.nodes())
initial_edges = len(G.edges())
avg_degree = np.mean([G.degree(node) for node in G.nodes()])
initial_connected_components= nx.number_connected_components(G)

print(f"Initial node count: {initial_node_count}")
print(f"Initial number of edges: {initial_edges}")
print(f"Average node degree: {avg_degree}")
print(f"Initial number of connected components: {initial_connected_components}")

Initial node count: 212
Initial number of edges: 242
Average node degree: 2.2830188679245285
Initial number of connected components: 19


<li><B>2 Προεργασία για την πρόβλεψη συνδέσμων. 2(i). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

In [5]:
# Υπολογισμός πίνακα γειτνίασης
adj_matrix = nx.to_numpy_array(G)
n = len(adj_matrix)

# Εύρεση ασύνδετων ζευγών
unconnected_pairs = []
for i in range(n):
  for j in range(i+1, n):
    if adj_matrix[i, j] == 0:
      unconnected_pairs.append((i+1, j+1)) # Η αρίθμηση κόμβων ξεκινάει από 1

# Υπολογισμός αριθμού ασύνδετων ζευγών
num_unconnected_pairs = len(unconnected_pairs)
print(f"Number of unconnected pairs: {num_unconnected_pairs}")

Number of unconnected pairs: 22124


<li><B>2(ii).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

In [6]:
removable_edges = []

# Έλεγχος κάθε ακμής
G_temp = G.copy()  # Δημιουργούμε ένα προσωρινό γράφο

for edge in list(G.edges()):
    G_temp.remove_edge(*edge)  # Αφαιρούμε την ακμή

    # Ελέγχουμε τις προϋποθέσεις
    if (
        nx.number_connected_components(G_temp) == initial_connected_components and
        G_temp.number_of_nodes() == initial_node_count
    ):
        removable_edges.append(edge)  # Αποθηκεύουμε την ακμή αν πληροί τις συνθήκες
    else:
        G_temp.add_edge(*edge) # Την ξαναπροσθέτουμε

# Αριθμός removable edges
num_removable_edges = len(removable_edges)

print(f"Number of removable edges: {num_removable_edges}")

Number of removable edges: 49


<li><B>2(iii).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 2(i) και 2(ii) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>2(iv).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 2(ii). <BR>
    <li><B>2(v).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

In [12]:
#dataframe of removable edges (they will function as positive samples)
removable_edges_df = pd.DataFrame(removable_edges, columns=['source', 'target'])
removable_edges_df['link'] = 1

#dataframe of unconnected node pairs (they will function as negative samples)
unconnected_pairs_df = pd.DataFrame(unconnected_pairs, columns=['source', 'target'])
unconnected_pairs_df['link'] = 0

#merge in one dataframe
combined_df = pd.concat([removable_edges_df, unconnected_pairs_df], ignore_index=True)

#build graph G'=G_reduced
G_reduced = G.copy()
G_reduced.remove_edges_from(removable_edges)

num_nodes = len(G_reduced.nodes())
num_edges = len(G_reduced.edges)
avg_degree = np.mean([G_reduced.degree(node) for node in G_reduced.nodes()])
num_components = nx.number_connected_components(G_reduced)

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Average degree: {avg_degree}")
print(f"Number of connected components: {num_components}")


Number of nodes: 212
Number of edges: 193
Average degree: 1.820754716981132
Number of connected components: 19


<html><font size=4><B>3. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>3(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων:
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

In [13]:
!pip install node2vec

In [21]:
from node2vec import Node2Vec
import multiprocessing

#generate walks
node2vec_walker = Node2Vec(
    G_reduced,
    dimensions=128,
    walk_length=80,
    num_walks=10,
    p=1.0,
    q=1.0,
    workers=multiprocessing.cpu_count()
)

#train node2vec model
n2v_model = node2vec_walker.fit(window=10, min_count=1)

Computing transition probabilities:   0%|          | 0/212 [00:00<?, ?it/s]

<html><font>
    <li><B>3(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>3(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

In [22]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(combined_df['source'], combined_df['target'])]

In [23]:
from sklearn.model_selection import train_test_split

# Στόχος (1-removable_edges ή 0-unconnected_pairs )
y = combined_df['link']

# Διαχωρισμός δεδομένων σε σύνολα εκπαίδευσης και δοκιμών
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Πρόβλεψη στο σύνολο δοκιμών
y_pred = classifier.predict(X_test)
# Υπολογισμός ακρίβειας
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9976551226551227
